参考：[【連載】LangChainの公式チュートリアルを1個ずつ地味に、地道にコツコツと【Basic編#2】](https://zenn.dev/chips0711/articles/0dd345d6c1e118)

### 基本的なチャットモデルの使用方法

 - メッセージの送信とそれに対するAIの応答実施例  
User「こんにちは!わたしはボブです!」  
AI「こんにちは、ボブさん！お元気ですか？今日はどんなことをお話ししましょうか？」  
 - この段階では、モデルは会話の状態を持たず、独立したメッセージに対してのみ応答を返す  

※参考サイトでは環境変数名が間違っているため、修正必要

In [3]:
import os

from dotenv import find_dotenv, load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 環境変数を設定します
load_dotenv(find_dotenv())

# AzureChatOpenAIインスタンスを作成します
model = AzureChatOpenAI(
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ["OPENAI_API_VERSION"],
    streaming=True
)

# シンプルなメッセージをモデルに渡してみましょう
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="こんにちわ!わたしはボブです!")])
print(response.content)

こんにちは、ボブさん！お元気ですか？今日はどんなことをお話ししましょうか？


### 会話履歴の管理

##### ライブラリの追加インポート必要

In [ ]:
#!pip install langchain_community

##### RunnableWithMessageHistory クラスの使用方法
RunnableWithMessageHistoryクラスは、別のRunnableオブジェクト（この場合は言語モデル）をラップし、そのチャットメッセージの履歴を管理する。  
具体的な動作は以下の通り：

1. 会話の前に、過去のメッセージをRunnableに渡す前にロード
2. 実行後に生成された応答をメッセージとして保存
3. session_idを使って複数の会話を管理し、呼び出し時にconfigでsession_idを指定して該当する会話履歴を読み込み

##### SQLiteを使用してメッセージ履歴を管理する例

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# セッションIDに基づいて履歴を取得する関数を定義します
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, connection="sqlite:///memory.db")

# メッセージ履歴を持つチャットボットを作成します
with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history,
)

# コンフィグを設定し、チャットボットにメッセージを送信します
response = with_message_history.invoke(
    [HumanMessage(content="こんにちは！私はボブです。")],
    config={"configurable": {"session_id": "1"}},
)

print(response.content)

こんにちは、ボブさん！再びお会いできて嬉しいです。今日はどんなことをお話ししましょうか？


In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="私の名前は何ですか？")],
    config={"configurable": {"session_id": "1"}},
)

print(response.content)

あなたの名前はボブさんです。何か他にお話ししたいことがありますか？


### プロンプトテンプレートの追加

 - プロンプトテンプレートは、ユーザーからの入力を特定のフォーマットに変換することで、LLMが処理しやすい形式に整えるためのもの
 - ChatPromptTemplateを使用することで、システムメッセージとユーザーメッセージを組み合わせた柔軟なプロンプトを作成できる
 - 以下のコードでは、システムメッセージを使って、チャットボットに「役立つアシスタント」としての役割を指示している
   - MessagesPlaceholderは、会話履歴を挿入する場所を指定するために使用されている
   - これにより、過去の会話コンテキストを保持しながら、新しい入力に対応することができる

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# プロンプトテンプレートを作成します
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは役に立つアシスタントです。全ての質問に対してできる限りの答えをしてください。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# チェーンを作成し、メッセージを送信します
chain = prompt | model | StrOutputParser()

response = chain.invoke({"messages": [HumanMessage(content="こんにちは！私はボブです。")]})
print(response)

こんにちは、ボブさん！お会いできて嬉しいです。今日はどんなことをお手伝いできますか？


### 多言語対応のチャットボット
 - ユーザーが選択した言語で応答するようにチャットボットを設定
 - NOTE: メッセージ送信の間隔が狭いと待ちがあるようで、実行に時間がかかる

In [11]:
# 多言語対応のプロンプトテンプレートを作成します
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは役に立つアシスタントです。全ての質問に対して{language}で答えてください。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# チェーンを作成し、メッセージを送信します
chain = prompt | model | StrOutputParser()

response = chain.invoke(
    {"messages": [HumanMessage(content="こんにちは！私はボブです。")], "language": "スペイン語"}
)
print(response)

¡Hola, Bob! ¿Cómo estás?


### カスタマイズオプション
 - ConfigurableFieldSpecを使用して、user_idとconversation_idの2つのパラメータで会話履歴を管理する方法
 - 同じユーザーIDであれば過去の会話履歴を保持し、新しいユーザーIDであれば新しい会話履歴が始まる
 - 複数のユーザーとの会話を個別に管理しつつ、言語設定も柔軟に変更できる高度なチャットボットシステムを構築できる

In [12]:
from langchain_core.runnables import ConfigurableFieldSpec

# プロンプトテンプレートを作成します
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは役に立つアシスタントです。全ての質問に対して{language}でできる限りの答えをしてください。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# プロンプトテンプレートとモデルを組み合わせたランナブルを作成します
runnable_with_prompt = prompt | model | StrOutputParser()

# セッションIDに基づいて履歴を取得する関数を定義します
def get_session_history(user_id: str, conversation_id: str):
    return SQLChatMessageHistory(f"{user_id}--{conversation_id}", connection="sqlite:///memory.db")

# メッセージ履歴を持つチャットボットを作成します
with_message_history = RunnableWithMessageHistory(
    runnable_with_prompt,
    get_session_history,
    input_messages_key="messages",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="ユーザーID",
            description="ユーザーのユニークな識別子。",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="会話ID",
            description="会話のユニークな識別子。",
            default="",
            is_shared=True,
        ),
    ],
)

# 正しい形式の入力メッセージと設定を行います
input_message = [HumanMessage(content="こんにちは！私はボブです。")]

# メッセージを送信して応答を取得します
response = with_message_history.invoke(
    {"language": "イタリア語", "messages": input_message},  
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

print(response)

Ciao Bob! Come posso aiutarti oggi?


日本語で名前を覚えているか確認
→　覚えていない

In [13]:
# 正しい形式の入力メッセージと設定を行います
input_message = [HumanMessage(content="私の名前は何ですか？")]

# メッセージを送信して応答を取得します
response = with_message_history.invoke(
    {"language": "日本語", "messages": input_message},  
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

print(response)

申し訳ありませんが、あなたの名前を知ることはできません。名前を教えていただければ、その名前でお呼びすることができますよ。


日本語で再度メッセージを送信

In [14]:
# 正しい形式の入力メッセージと設定を行います
input_message = [HumanMessage(content="こんにちは！私はボブです。")]

# メッセージを送信して応答を取得します
response = with_message_history.invoke(
    {"language": "日本語", "messages": input_message},  
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

print(response)

こんにちは、ボブさん！お会いできて嬉しいです。今日はどんなことについてお話ししましょうか？


再度、日本語で名前を覚えているか確認　→　記憶できていない・・・  
session_idでの履歴管理は出来ていたが何故？

In [15]:
# 正しい形式の入力メッセージと設定を行います
input_message = [HumanMessage(content="私の名前は何ですか？")]

# メッセージを送信して応答を取得します
response = with_message_history.invoke(
    {"language": "日本語", "messages": input_message},  
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

print(response)

申し訳ありませんが、あなたの名前はわかりません。お名前を教えていただければ、何かお手伝いできるかもしれません。


In [16]:
# 正しい形式の入力メッセージと設定を行います
input_message = [HumanMessage(content="私の名前は何ですか？")]

# メッセージを送信して応答を取得します
response = with_message_history.invoke(
    {"language": "日本語", "messages": input_message},  
    config={"configurable": {"user_id": "123", "conversation_id": "2"}},
)

print(response)

申し訳ありませんが、あなたの名前はわかりません。私に名前を教えていただければ、呼び方を工夫することができますよ！


### 会話履歴の管理と最適化
 - チャットボットを長期間運用する場合、会話履歴が無限に増え続けると、LLMのコンテキストウィンドウを超えてしまい、効率的な応答ができなくなる可能性がある
 - LangChainには、会話履歴のサイズを適切に制限するためのツールが用意されている

##### メッセージ履歴のトリミング
 - trim_messages ユーティリティを使用すると、指定されたトークン数に合わせて履歴をトリミングできる
 - これにより、モデルのコンテキストウィンドウ内に収まるようにメッセージの長さを調整できる

In [17]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages
from langchain_openai import AzureChatOpenAI

# トリミング設定を作成します
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=AzureChatOpenAI(model="gpt-4o"),
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# メッセージリストを定義します
messages = [
    SystemMessage(content="あなたは優秀なアシスタントです。"),
    HumanMessage(content="こんにちは！私はボブです。"),
    AIMessage(content="こんにちは！"),
    HumanMessage(content="私はバニラアイスクリームが好きです。"),
    AIMessage(content="いいですね。"),
    HumanMessage(content="2 + 2は何ですか？"),
    AIMessage(content="4です。"),
    HumanMessage(content="ありがとう。"),
    AIMessage(content="どういたしまして！"),
    HumanMessage(content="楽しんでますか？"),
    AIMessage(content="はい！"),
]

# メッセージ履歴をトリミングします
trimmed_messages = trimmer.invoke(messages)
print(trimmed_messages)

[SystemMessage(content='あなたは優秀なアシスタントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='ありがとう。', additional_kwargs={}, response_metadata={}), AIMessage(content='どういたしまして！', additional_kwargs={}, response_metadata={}), HumanMessage(content='楽しんでますか？', additional_kwargs={}, response_metadata={}), AIMessage(content='はい！', additional_kwargs={}, response_metadata={})]


 - 以下はトリミングされた履歴を使ってプロンプトにメッセージを渡す例
 - 結果を見ると、チャットボットは名前を知らないと正直に答えている
 - これは、トリミングによって以前の会話（名前の紹介を含む）が削除されたため

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# プロンプトテンプレートを定義します
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは役に立つアシスタントです。"),
        ("human", "{question}"),
    ]
)

# トリミング結果を表示する関数
def print_trimmed(messages):
    print("Trimmed messages:")
    for msg in messages:
        print(f"{msg.type}: {msg.content}")
    return messages

# メッセージ履歴をトリミングしてからプロンプトに渡すチェーンを作成します
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer | print_trimmed)
    | prompt
    | model
)

# 入力メッセージを作成
input_messages = messages + [HumanMessage(content="私の名前は何ですか？")]

response = chain.invoke(
    {
        "messages": input_messages,
        "question": "私の名前は何ですか？",
        "language": "日本語",
    }
)
print("\nResponse:")
print(response.content)

Trimmed messages:
system: あなたは優秀なアシスタントです。
human: ありがとう。
ai: どういたしまして！
human: 楽しんでますか？
ai: はい！
human: 私の名前は何ですか？

Response:
申し訳ありませんが、あなたの名前はわかりません。あなたの名前を教えていただければ、その名前でお呼びすることができます。


trimmerの設定を変えたらどうか  
→　やはりメモリ機能が効いていない

In [27]:
# 古い履歴を残すトリミング設定を作成
trimmer = trim_messages(
    max_tokens=65,
    strategy="first",
    token_counter=AzureChatOpenAI(model="gpt-4o"),
    # include_system=False,
    allow_partial=False,
    # start_on="human",
)

# メッセージ履歴をトリミングしてからプロンプトに渡すチェーンを作成します
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer | print_trimmed)
    | prompt
    | model
)

# 入力メッセージを作成
input_messages = messages[1:] + [HumanMessage(content="私の名前は何ですか？")]

response = chain.invoke(
    {
        "messages": input_messages,
        "question": "私の名前は何ですか？",
        "language": "日本語",
    }
)
print("\nResponse:")
print(response.content)

Trimmed messages:
human: こんにちは！私はボブです。
ai: こんにちは！
human: 私はバニラアイスクリームが好きです。
ai: いいですね。
human: 2 + 2は何ですか？
ai: 4です。

Response:
申し訳ありませんが、あなたの名前はわかりません。あなたの名前を教えていただければ、それに基づいてお話しすることができますよ。


##### チャット履歴とトリミングの組み合わせ
 - RunnableWithMessageHistoryを使って、トリミングされたメッセージ履歴を管理する方法
 - ユーザーとの会話履歴を保持しつつ、適切にトリミングされたデータのみをプロンプトに渡すことができる

In [28]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# プロンプトテンプレートの作成
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは役に立つアシスタントです。全ての質問に対して{language}で答えてください。"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# チェーンの作成
chain = (
    RunnablePassthrough.assign(history=lambda x: x["history"])
    | prompt
    | model
    | StrOutputParser()
)

# チャット履歴の初期化
messages = [
    SystemMessage(content="あなたは優秀なアシスタントです。"),
    HumanMessage(content="こんにちは！私はボブです。"),
    AIMessage(content="こんにちは、ボブさん！お手伝いできることがありますか？"),
]
chat_history = InMemoryChatMessageHistory(messages=messages)

# セッションIDに基づいて履歴を取得するダミー関数を定義
def dummy_get_session_history(session_id):
    if session_id != "1":
        return InMemoryChatMessageHistory()
    return chat_history

# トリミングされたメッセージ履歴を持つチェーンを作成
chain_with_history = RunnableWithMessageHistory(
    chain,
    dummy_get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# メッセージを送信して応答を取得
response = chain_with_history.invoke(
    {
        "input": "喋れないオウムを何と呼びますか？また､私の名前はなんですか?",
        "language": "日本語"
    },
    config={"configurable": {"session_id": "1"}},
)

print(response)

喋れないオウムは「無口のオウム」や「黙ったオウム」と呼ぶことができます。また、あなたの名前はボブですね！他に知りたいことがあれば教えてください。


セッションIDを変更してみる  
→　新しいセッションIDでは、以前の会話履歴（ボブさんの名前を含む）にアクセスできていないことがわかる  
　　これは、会話履歴が適切にセッションごとに分離されていることを示している

メモリ周りの動作参考サイト
 - [LangChain の Memory の概要 ](https://note.com/npaka/n/nbd04bdc041cb)
 - [LCEL記法のChainにMemoryを組み込む方法](https://zenn.dev/mizunny/articles/d974720d8acc6f)

In [29]:
response = chain_with_history.invoke(
    {
        "input": "喋れないオウムを何と呼びますか？また､私の名前はなんですか?",
        "language": "日本語"
    },
    config={"configurable": {"session_id": "2"}},
)

print(response)

喋れないオウムは「無口のオウム」や「黙っているオウム」と呼ぶことができます。また、あなたの名前については、私には分かりません。もし教えていただければ、その名前でお呼びすることができます。


### ストリーミングの導入
 - リアルタイムで生成されるトークンを見ることができ、よりスムーズなインタラクションを可能とする機能
 - ユーザーは応答の完了を待つことなく、途中経過を確認できる

##### 1. 同期的なストリーミング（stream メソッド）
 - 常のPythonコードで使用する
 - 以下のコードでは、chain_with_history.streamメソッドを使用して、チャットモデルからのストリーミング応答を逐次取得している
 - 実際のアプリケーションでは、end="|" を end="" に変更することで、ユーザーにはスムーズな文字の流れとして表示される 
 - ここでは、ストリーミングの過程を可視化するために | で区切って表示している。

In [30]:
from langchain_core.messages import HumanMessage

config = {
    "configurable": {
        "session_id": "abc15",
        "user_id": "user123"
    }
}

# ストリーミング応答を取得します
for r in chain_with_history.stream(
    {
        "input": "こんにちは！私はトッドです。ジョークを教えてください",
        "language": "日本語",
    },
    config=config,
):
    print(r, end="|", flush=True)
print()  # 最後に改行を入れる

||こんにちは|、|ト|ッド|さん|！|ジョ|ーク|を|一|つ|お|教|え|します|ね|。

|「|カ|メ|と|ウ|サ|ギ|が|レ|ース|を|しました|。|カ|メ|は|遅|い|け|ど|、|ウ|サ|ギ|は|早|く|走|り|す|ぎ|て|寝|ちゃ|った|。|カ|メ|が|勝|った|ん|だけ|ど|、|ウ|サ|ギ|は|こう|言|いました|。
|『|次|は|寝|ない|よう|に|する|よ|！|』|」

|どう|でした|か|？|笑|って|いただ|け|た|ら|嬉|しい|です|！||


##### 2.非同期ストリーミング（astream メソッド）
 - asyncioなどの非同期プログラミング環境で使用する
 - Webアプリケーションなど、非同期処理が必要な環境で特に有用
 - 他の非同期タスクと並行して効率的に処理を行うことができる
 - 参考サイトのコードそのままでは非同期処理の実行でエラーでるため注意
   - await関数で直接実行すれば問題なく動作([参考](https://qiita.com/osorezugoing/items/d26921f0affd62b87858))

In [ ]:
import asyncio

async def get_streaming_response():
    async for r in chain_with_history.astream(
        {
            "input": "別のジョークを教えてください。今度は猫に関するものをお願いします。",
            "language": "日本語",
        },
        config=config,
    ):
        print(r, end="|", flush=True)
    print()  # 最後に改行を入れる

# 非同期関数を実行
# asyncio.run(get_streaming_response()) # NOTE:asyncio.run() cannot be called from a running event loop
await get_streaming_response()

||もちろん|です|！|こん|な|猫|の|ジョ|ーク|はい|か|が|です|か|？

|「|猫|が|コン|ピ|ュー|ター|を|使|う|と|き|、|何|を|最|初|に|する|と思|います|か|？」

|「|マ|ウ|ス|を|捕|ま|える|こと|！」| 

|どう|でした|か|？|少|し|笑|って|いただ|け|た|ら|嬉|しい|です|！||


##### エラーハンドリング
 - 以下は、try-exceptブロックを使用してエラーをキャッチする例
 - ストリーミング中にエラーが発生しても、プログラムが予期せず停止することを防ぐ

In [34]:
async def handle_streaming_response():
    try:
        async for r in chain_with_history.astream(
            {
                "input": "プログラミングに関する面白い事実を教えてください",
                "language": "日本語",
            },
            config=config,
        ):
            print(r, end="", flush=True)
        print()  # 最後に改行を入れる
    except Exception as e:
        print(f"エラーが発生しました: {e}")

await handle_streaming_response()


プログラミングに関する面白い事実の一つは、最初のプログラマーとされるアダ・ラブレス（Ada Lovelace）です。彼女は1830年代にチャールズ・バベッジの解析機関（Analytical Engine）に関するノートを作成し、その中で初めてコンピュータプログラムを書いたとされています。ラブレスは、計算機が単なる計算以上のことを実行できる可能性を見出し、後のコンピュータ科学に大きな影響を与えました。

さらに、彼女の名前を冠したプログラミング言語「Ada」も存在します。これは、航空宇宙や軍事などの分野で広く使用されている高級言語です。アダ・ラブレスの存在は、女性がテクノロジーの分野で果たす重要な役割を示す象徴的な例でもあります。
